# This notebook hypothesizes that broken masks found in the dataset are caused by incorrect polygon->bitmask conversion

This was originally found by @bakeryproducts 
I hadn't seen his notebook before it was removed, but I imagine it went something like this

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Here is a sample cell

In [ ]:
img = cv2.imread('../input/sartorius-cell-instance-segmentation/train/13325f865bb0.png')
_=plt.imshow(img[50:150, 160:310])


## Here is a hypotetical polygon annotation of that cell

In [ ]:
mask = np.load('../input/sample-mask/mask.npy')
cont, hier = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
_=plt.imshow(cv2.drawContours(np.zeros_like(mask), cont, -1, 1, 1))

## Here is the polygon above converted to bitmask with `cv2.fillConvexPoly` - leading to a clearly broken mask

In [ ]:
convex_mask = cv2.fillConvexPoly(np.zeros_like(mask),points=cont[0], color=1)
_=plt.imshow(convex_mask)

In [ ]:
def rle_decode(mask_rle, shape):
    s = mask_rle.split()    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    img = np.zeros((shape[0] * shape[1]), dtype=np.uint8)
    for start, end in zip(starts, ends):
        img[start : end] = 1
    return img.reshape(shape)

## And here is the actual mask found in the competition training data

In [ ]:
df_train = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
label = df_train.iloc[4899]["annotation"]
mask = rle_decode(label, shape=(520, 704))
_=plt.imshow(mask[50:150, 160:310])